In [69]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import os

In [62]:
request = requests.get('https://adindex.ru/ratings/marketing/2021/300770/#300773')
html = BS(request.content, 'html.parser')

In [63]:
table = html.find('table', attrs={'class': 'rating-table-full'})

In [64]:
def get_data_table(table):
    table_data_array = []
    for row in table.find_all("tr"):
        cells_name = row.find_all("th")
        array_text_cells = []
        if cells_name is not None:
            for cell_name in cells_name:
                array_text_cells.append(cell_name.text)
        cells = row.find_all("td")
        for cell in cells:
            if cell.find("a") is not None:
                text_cell = cell.find("a").text
            else:
                text_cell = cell.text
                try:
                    text_cell = float(cell.text.replace(" ", "").replace(",", "."))
                except ValueError:
                    text_cell = cell.text
                    if text_cell.isspace():
                        text_cell = None
            if text_cell == 'N/a' or text_cell == '-' or text_cell == '':
                text_cell = None
            array_text_cells.append(text_cell)
        if not array_text_cells:
            continue
        table_data_array.append(array_text_cells)
    return table_data_array

In [65]:
def array_to_df(data):
    columns = data.pop(0)
    return pd.DataFrame(data, columns=columns)

In [66]:
def get_sheets_name():
    name_array = []
    div_names = html.find('div', attrs={'class': 'rating-tabs-categories__index'})
    for cell in div_names:
        text = cell.text
        if text != '\n':
            name_array.append(text)
    return name_array

In [67]:
import openpyxl
def main():
    file_name = 'tables.xlsx'
    tables = html.find_all('table', attrs={'class': 'rating-table-full'})
    writer = pd.ExcelWriter(file_name)
    sheets_name = get_sheets_name()
    for i, table in enumerate(tables):
        data = get_data_table(table)
        df = array_to_df(data)
        if i == 0:
#             df.fillna(0, inplace=True)
            df['Средняя прибыль за целые года'] = df[['2016 год','2017 год','2018 год', '2019 год', '2020 год', '2021 год']].mean(axis=1)
            df = df.sort_values(by='Средняя прибыль за целые года', ascending=False)
#         XL не дает делать название листа длинее 31-го символа. Индекс добален так как, 
#         если обрезать название, то получается два одинаковых.
        df.to_excel(writer, sheet_name=f'{sheets_name[i][:30]}{i}', index=False)
    writer.close()

In [68]:
main()